# connect to rds database

In [68]:
import pg8000.native
import boto3
import json
from faker import Faker

# 1. SETUP: Connect to your DB (use your Jupyter credentials)
conn = pg8000.native.Connection(user="", password="!", host="...", database="") #removed PII
fake = Faker()

# 2. GENERATE: Create 10 fake document logs
for _ in range(10):
    mock_doc = {
        "user_id": fake.uuid4(),
        "score": 0.95 if _ < 8 else 0.30, # Simulate 2 "Failures"
        "doc_type": "Passport"
    }
    conn.run("INSERT INTO audit_logs (raw_data) VALUES (:data)", data=json.dumps(mock_doc))

# 3. AUDIT: Read the data, ask Bedrock, and update the table
rows = conn.run("SELECT id, raw_data FROM audit_logs WHERE ai_status IS NULL")

for row in rows:
    row_id, data = row[0], row[1]
    # (Insert your Bedrock invoke_model logic here)
    # ai_result = "PASS" if score > 0.5 else "FAIL"
    
    conn.run("UPDATE audit_logs SET ai_status = :status WHERE id = :id", status=ai_result, id=row_id)

print("Local Audit Complete!")

✅ Connection successful!
Database Version: PostgreSQL 17.6 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit


# connect to bedrock

In [72]:
import boto3
import json

# This client talks to the Lambda service, not the DB
lambda_client = boto3.client('lambda', region_name='us-east-2')

# Define the sensor scenario you want to test
my_request = {
    "data": {
        "subject_id": "STRESS_TEST_01",
        "rgb": 0.15, 
        "ir": 0.95, 
        "glare": True
    }
}

# TRIGGER: This line sends the data to the cloud and waits for the result
response = lambda_client.invoke(
    FunctionName='Biometric-Audit-Processor', # Must match your Lambda's name
    Payload=json.dumps(my_request)
)

print("Result from Cloud:", json.loads(response['Payload'].read()))

Result from Cloud: {'errorMessage': "Can't create a connection to host biometric-audit.ctuwk6uyw8t6.us-east-2.rds.amazonaws.com and port 5432 (timeout is None and source_address is None).", 'errorType': 'InterfaceError', 'requestId': '5696a783-bcb9-4973-bd84-26e5ae0a3d4e', 'stackTrace': ['  File "/var/task/lambda_function.py", line 33, in lambda_handler\n    conn = pg8000.native.Connection(\n', '  File "/opt/python/pg8000/native.py", line 216, in __init__\n    super().__init__(*args, **kwargs)\n', '  File "/opt/python/pg8000/core.py", line 328, in __init__\n    self.channel_binding, self._usock = _make_socket(\n', '  File "/opt/python/pg8000/core.py", line 206, in _make_socket\n    raise InterfaceError(\n']}


Visualize Performance of AI Auditer

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Pull the results into a DataFrame
df = pd.DataFrame(conn.run("SELECT ai_status FROM audit_logs"), columns=['status'])

# Visualize the audit outcomes
df['status'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.ylabel("Number of Documents")
# Note: No title per your preference
plt.show()